In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Topic: EX2 - Turbofan RUL Prediction
**Task**: Predict the remaining useful life (RUL) of turbofan engines based on given sensor data (time series data). It is a forcasting problem, where the goal is to predict the number of cycles an engine will last before it fails.
**Data**: Turbofan engine degradation simulation data (NASA) - [Link](https://data.nasa.gov/dataset/Turbofan-Engine-Degradation-Simulation-Data-Set/vrks-gjie). See also in the topic [introduction notebook](https://github.com/nina-prog/damage-propagation-modeling/blob/2fb8c1a1102a48d7abbf04e4031807790a913a99/notebooks/Turbofan%20remaining%20useful%20life%20Prediction.ipynb).

**Subtasks**:
1. Perform a deep **exploratory data analysis (EDA)** on the given data.
2. Implement a more efficient **sliding window method** for time series data analysis. -> 🎯 **Focus on this task**
3. Apply **traditional machine learning methods** (SOTA) to predict the remaining useful life. Includes data preparation, feature extraction, feature selection, model selection, and model parameter optimization.
4. Create **neural network models** to predict the remaining useful life. Includes different architectures like Convolutional Neural Networks (CNN), Recurrent Neural Networks (RNN), or Attention Models. Note: You can search for SOTA research papers and reproduce current state-of-the-art models.


# Imports + Settings

In [21]:
!pip install pytorch-lightning
!pip install colorlog
!pip install bayesian-optimization
!pip install pandas
!pip install plotly
!pip install matplotlib
!pip install seaborn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [22]:
# third-party libraries
import pandas as pd
import numpy as np
import os
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from bayes_opt import BayesianOptimization

import time

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
# source code
from src.data_loading import load_data, load_config
from src.data_splitting import train_val_split_by_group
from src.nn_utils import create_sliding_window, create_sliding_window_test
from src.rolling_window_creator import calculate_RUL
from src.data_processing import apply_padding_on_train_data_and_test_data, drop_samples_with_clipped_values
from src.nn_util.nn_models.ligthning.cnnModel2 import CNNModel2 as CNNModel
from src.nn_util.nn_models.ligthning.cnnModel1 import CNNModel1 as CNNModel1
from src.nn_util.nn_models.ligthning.exampleLSTMModel import ExampleLSTMModel as LSTMModel
from src.nn_util.datamodule.lightning.turbofanDatamodule import TurbofanDatamodule
from src.data_cleaning import clean_data

In [26]:
# settings
sns.set_style("whitegrid")
sns.set_palette("Set2")
sns.set(rc={"figure.dpi":100, 'savefig.dpi':200})
sns.set_context('notebook')

In [27]:
np.random.seed(42)

# Paths

In [28]:
PATH_TO_CONFIG = "configs/config.yaml"

# Load Config + Data

In [29]:
config = load_config(PATH_TO_CONFIG) # config is dict

In [30]:
train_data, test_data, test_RUL_data = load_data(config_path=PATH_TO_CONFIG, dataset_num=4)

2024-06-01 05:40:01 [src.data_loading:43] [INFO] >>>> Loading data set 4...
2024-06-01 05:40:01 [src.data_loading:72] [INFO] >>>> Loaded raw data for dataset 4.
2024-06-01 05:40:01 [src.data_loading:73] [INFO] >>>> Train Data: (61249, 26)
2024-06-01 05:40:01 [src.data_loading:74] [INFO] >>>> Test Data: (41214, 26)
2024-06-01 05:40:01 [src.data_loading:75] [INFO] >>>> Test RUL Data: (248, 1)


# Hyperparameter Search for CNNModel1 for dataset FD004

First search space:
* learning rate initialisation: (0.00001, 0.1)
* batch size: (10, 200)
* dropout: (0.01, 0.5)
* betas (of classifier Adam): beta_1 (0.8, 0.999) and beta_2 (0.9, 0.999999)

Second search space:
* learning rate initialisation:  (0.001, 0.01)
* batch size: (140, 160)
* dropout: (0.01, 0.5)
* betas (of classifier Adam): beta_1 (0.9, 0.999) and beta_2 (0.9, 0.999)

Iterations: 26 for the first search space and 40 for the second search space 

Note: Due to an error in the first search space the run ended with the 26th iteration. The second search space was selected around the hyperparameters of the best model of the first search space.

In [31]:
# some hyperparameters
time_column = 'Cycle'
group_column = 'UnitNumber'

window_size = 30
clip_value = 125
apply_data_cleaning = True

# Apply scaler. The order in the list represents the order in which they are applied
std_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()
robust_scaler = RobustScaler()
scaler = [std_scaler, minmax_scaler, robust_scaler]

# The model_type can be 'CNN_2D' or 'CNN_1D' and defines what type of convolutional layers has been used.
model_type = 'CNN_1D' 

# Setting the seed
pl.seed_everything(21)

Seed set to 21


21

In [32]:
if apply_data_cleaning:
    train_data, test_data = clean_data(train_data, test_data, method=None, ignore_columns=['UnitNumber', 'Cycle'], threshold_missing=0.1, threshold_corr=0.0, contamination=0.05)
    
# Add column RUL to train_data
train_data = calculate_RUL(train_data, time_column, group_column, clip_value)

train_data, test_data = apply_padding_on_train_data_and_test_data(train_data=train_data, test_data=test_data, window_size=window_size)

X_test, _ = create_sliding_window_test(test_data, column_RUL=False, drop_columns=['UnitNumber'])
y_test = test_RUL_data.values

train, val = train_val_split_by_group(train_data, test_size=0.1, random_state=12)

X_train, y_train = create_sliding_window(train, window_size=window_size)  #, drop_columns=['UnitNumber', 'Cycle, 'RUL'])
X_val, y_val = create_sliding_window(val, window_size=window_size)  #, drop_columns=['UnitNumber', 'Cycle', 'RUL'])

X_train, y_train = drop_samples_with_clipped_values(X_train, y_train, clip_value)
X_val, y_val = drop_samples_with_clipped_values(X_val, y_val, clip_value)

2024-06-01 05:40:04 [src.data_cleaning:134] [INFO] >>>> Cleaning train and test data...
2024-06-01 05:40:04 [src.data_cleaning:136] [INFO] >>>> Formatting column types...
2024-06-01 05:40:04 [src.data_cleaning:69] [DEBUG] >>>> Found 0 categorical columns: []
2024-06-01 05:40:04 [src.data_cleaning:69] [DEBUG] >>>> Found 0 categorical columns: []
2024-06-01 05:40:04 [src.data_cleaning:141] [INFO] >>>> Handling duplicates...
2024-06-01 05:40:04 [src.data_cleaning:146] [INFO] >>>> Removing outliers...
2024-06-01 05:40:04 [src.outlier_detection:150] [DEBUG] >>>> Removing outliers using method: None ...
2024-06-01 05:40:04 [src.outlier_detection:162] [INFO] >>>> No outlier detection method specified. Skipping outlier detection.
2024-06-01 05:40:04 [src.outlier_detection:150] [DEBUG] >>>> Removing outliers using method: None ...
2024-06-01 05:40:04 [src.outlier_detection:162] [INFO] >>>> No outlier detection method specified. Skipping outlier detection.
2024-06-01 05:40:04 [src.data_cleaning:

Scale the data

In [33]:
X_train_shape = X_train.shape
X_val_shape = X_val.shape
X_test_shape = X_test.shape

# Note: Do not normalize the cycle value! That is why we start with one
for single_scaler in scaler:
    for i in range(1, X_train.shape[-1]):
        X_train[:, :, i] = single_scaler.fit_transform(X_train[:, :, i])
        X_val[:, :, i] = single_scaler.transform(X_val[:, :, i])
        X_test[:, :, i] = single_scaler.transform(X_test[:, :, i])

Change data types of arrays to float32 and swap axes if necessary:

In [34]:
model_type = 'CNN_1D'

print(X_train.shape)
if model_type == 'CNN_1D':
    X_train = np.swapaxes(X_train, 1, 2)
elif model_type == 'CNN_2D':
    X_train = np.swapaxes(X_train, 1, 2)
    X_train = X_train[:, np.newaxis, :, :]
X_train = np.array(X_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)
print(X_train.shape)

print(X_val.shape)
if model_type == 'CNN_1D':
    X_val = np.swapaxes(X_val, 1, 2)
elif model_type == 'CNN_2D':
    X_val = np.swapaxes(X_val, 1, 2)
    X_val = X_val[:, np.newaxis, :, :]
X_val = np.array(X_val, dtype=np.float32)
y_val = np.array(y_val, dtype=np.float32)
print(X_val.shape)

print(X_test.shape)
if model_type == 'CNN_1D':
    X_test = np.swapaxes(X_test, 1, 2)
elif model_type == 'CNN_2D':
    X_test = np.swapaxes(X_test, 1, 2)
    X_test = X_test[:, np.newaxis, :, :]
X_test = np.array(X_test, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)
print(X_test.shape)

(28188, 30, 25)
(28188, 25, 30)
(3150, 30, 25)
(3150, 25, 30)
(248, 30, 25)
(248, 25, 30)


## Hyperparameter Search 

In [ ]:
from sklearn.metrics import root_mean_squared_error
import torch 
# Setting the seed
pl.seed_everything(21)

def hyperparameter_function_cnn(learning_rate_init, batch_size, dropout, beta_1, beta_2):
    """ Function for hyperparameter optimization
    """    
    # Select hyperparameters of trainer!
    checkpoint_callback = ModelCheckpoint(monitor="val_loss")
    trainer = Trainer(min_epochs=1, max_epochs=150, callbacks=[checkpoint_callback], deterministic=True)
    datamodule = TurbofanDatamodule(batch_size=int(batch_size), random_state=15)
    datamodule.set_train_dataset(X_train, y_train)
    datamodule.set_val_dataset(X_val, y_val)
    datamodule.set_predict_dataset(X_test)
    datamodule.set_test_dataset(X_test, y_test)
    model = CNNModel(lr=learning_rate_init, window_size=window_size, features=25, dropout_rate=dropout, beta_1=beta_1, beta_2=beta_2)
    trainer.fit(model, datamodule=datamodule)
    pred = trainer.validate(model, datamodule=datamodule, ckpt_path='best')
    return - pred[0]['val_loss']
    

# Bounded region of parameter space
pbounds = {'learning_rate_init': (0.00001, 0.1), 'batch_size': (10, 200), 'dropout': (0.01, 0.5), 'beta_1': (0.8, 0.999), 'beta_2': (0.9, 0.999999)}

optimizer = BayesianOptimization(
    f=hyperparameter_function_cnn,
    pbounds=pbounds,
    random_state=15,
)
optimizer.maximize(
    init_points=15,
    n_iter=25,
)

In [16]:
for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

Iteration 0: 
	{'target': -1180.4991455078125, 'params': {'batch_size': 171.27536248102996, 'beta_1': 0.8356002890592787, 'beta_2': 0.9054362670674289, 'dropout': 0.18715383858322926, 'learning_rate_init': 0.027547338851355133}}
Iteration 1: 
	{'target': -888.2766723632812, 'params': {'batch_size': 110.7000427301308, 'beta_1': 0.8608778642314473, 'beta_2': 0.9304471314410552, 'dropout': 0.0647532253260414, 'learning_rate_init': 0.02499740239787265}}
Iteration 2: 
	{'target': -1321.932373046875, 'params': {'batch_size': 184.34968058894685, 'beta_1': 0.8525652237974214, 'beta_2': 0.9717766509736682, 'dropout': 0.43420036675317547, 'learning_rate_init': 0.08070987742971512}}
Iteration 3: 
	{'target': -1051.4298095703125, 'params': {'batch_size': 50.00461070517731, 'beta_1': 0.8332813632598327, 'beta_2': 0.9046705924618155, 'dropout': 0.029316932761174072, 'learning_rate_init': 0.020031078669449538}}
Iteration 4: 
	{'target': -1321.932373046875, 'params': {'batch_size': 199.72324651816268,

In [ ]:
from sklearn.metrics import root_mean_squared_error
import torch 
# Setting the seed
pl.seed_everything(21)

def hyperparameter_function_cnn(learning_rate_init, batch_size, dropout, beta_1, beta_2):
    """ Function for hyperparameter optimization
    """    
    # Select hyperparameters of trainer!
    checkpoint_callback = ModelCheckpoint(monitor="val_loss")
    trainer = Trainer(min_epochs=1, max_epochs=150, callbacks=[checkpoint_callback], deterministic=True)
    datamodule = TurbofanDatamodule(batch_size=int(batch_size), random_state=15)
    datamodule.set_train_dataset(X_train, y_train)
    datamodule.set_val_dataset(X_val, y_val)
    datamodule.set_predict_dataset(X_test)
    datamodule.set_test_dataset(X_test, y_test)
    model = CNNModel(lr=learning_rate_init, window_size=window_size, features=25, dropout_rate=dropout, beta_1=beta_1, beta_2=beta_2)
    trainer.fit(model, datamodule=datamodule)
    pred = trainer.validate(model, datamodule=datamodule, ckpt_path='best')
    return - pred[0]['val_loss']
    

# Bounded region of parameter space
pbounds = {'learning_rate_init': (0.001, 0.01), 'batch_size': (140, 160), 'dropout': (0.01, 0.2), 'beta_1': (0.9, 0.999), 'beta_2': (0.9, 0.999)}

optimizer = BayesianOptimization(
    f=hyperparameter_function_cnn,
    pbounds=pbounds,
    random_state=20,
)
optimizer.maximize(
    init_points=15,
    n_iter=25,
)

In [36]:
for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

Iteration 0: 
	{'target': -578.6076049804688, 'params': {'batch_size': 151.76261602154548, 'beta_1': 0.9888736590630324, 'beta_2': 0.9882615422179961, 'dropout': 0.16500912068845996, 'learning_rate_init': 0.0013230062705516796}}
Iteration 1: 
	{'target': -620.138916015625, 'params': {'batch_size': 153.83515163517768, 'beta_1': 0.9374894132585354, 'beta_2': 0.95133258359936, 'dropout': 0.13501077845617449, 'learning_rate_init': 0.002744651960946535}}
Iteration 2: 
	{'target': -505.94805908203125, 'params': {'batch_size': 145.44632804133167, 'beta_1': 0.9711419874265155, 'beta_2': 0.9775173573364421, 'dropout': 0.17156225155724913, 'learning_rate_init': 0.007977204045949265}}
Iteration 3: 
	{'target': -614.5155639648438, 'params': {'batch_size': 140.73328612842172, 'beta_1': 0.9115526797781567, 'beta_2': 0.9743767892492824, 'dropout': 0.055451461077925805, 'learning_rate_init': 0.003293254125479321}}
Iteration 4: 
	{'target': -1158.251708984375, 'params': {'batch_size': 157.1525106233952

In [37]:
max_params = optimizer.max['params']
print(optimizer.max)

{'target': -434.22528076171875, 'params': {'batch_size': 153.588222351065, 'beta_1': 0.9644278054982097, 'beta_2': 0.926610728635691, 'dropout': 0.02279168671841337, 'learning_rate_init': 0.007943006245227067}}
